In [24]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
import folium
import networkx as nx

In [25]:
df = pd.read_excel('상위 10퍼 정류장_최종.xlsx')
df

,정류소명,위도,경도,cluster
0,하귀리,126.395170,33.479550,4
1,사평마을,126.512379,33.492870,0
2,아라동주민센터,126.544900,33.475900,5
3,월구마을,126.507000,33.500133,0
4,함덕광명 샤인빌앞,126.674617,33.538233,1
...,...,...,...,...
58,탐라도서관,126.477589,33.477538,0
59,윗가시나물,126.569655,33.467351,1
60,선돌목동길,126.550808,33.449125,1
61,브릭캠퍼스,126.486001,33.458275,0


In [26]:
# 각 클러스터별로 TSP 문제 해결 및 원형 경로 생성
def solve_tsp(coords):
    # 거리 행렬 계산
    dist_matrix = cdist(coords, coords, metric='euclidean')
    
    # 그래프 생성
    G = nx.from_numpy_array(dist_matrix)
    
    # 최소 비용의 TSP 경로 찾기
    tsp_path = nx.approximation.traveling_salesman_problem(G, cycle=True)
    
    ordered_coords = coords[tsp_path]
    
    return ordered_coords

# 결과 저장
results = {}

# 클러스터별로 TSP 해결 및 원형 경로 생성
for cluster in df['cluster'].unique():
    cluster_coords = df[df['cluster'] == cluster][['위도', '경도']].values
    if len(cluster_coords) < 2:
        continue  # 클러스터에 데이터가 없거나 하나만 있으면 건너뛰기
    tsp_route = solve_tsp(cluster_coords)
    results[cluster] = tsp_route

# 지도 시각화
map_ = folium.Map(location=[33.5, 126.5], zoom_start=11)

# 클러스터별로 경로를 지도에 추가
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue']

for cluster, route in results.items():
    # 경로를 폴리라인으로 변환
    route_line = [(coord[1], coord[0]) for coord in route]  # 위도와 경도를 (위도, 경도)로 변환
    folium.PolyLine(route_line, color=colors[cluster % len(colors)], weight=2.5, opacity=1).add_to(map_)
    
    # 각 정류소에 마커 추가
    for coord in route:
        folium.Marker(location=[coord[1], coord[0]], icon=folium.Icon(color=colors[cluster % len(colors)])).add_to(map_)

# 지도 저장
map_.save("tsp_routes.html")
map_